# SDL4Kids.com: Your first self-driving laboratory (MacOS version)

This notebook helps you run your first SDL, using the BBC Micro:bit, see [sdl4kids.com](http://sdl4kids.com).

Go to [sdl4kids.com](http://sdl4kids.com) for the Windows version.

If you have any questions, check out the [FAQ](https://sites.google.com/matterhorn.studio/sdl4kids/faq)!

## Introduction

In this notebook, you will run your first closed-loop SDL.

Every SDL has three core components

1. **Synthesis**: We will use your computer screen to *generate rectangles with different RGB colors*.
2. **Characterisation**: We will use the Microbit to *measure the RGB color* on the screen. 
3. **Planning**: We will use different *search strategies to find the correct RGB color* combination.

## (1) Synthesis: Using your screen as an RGB color generator

**Goal**: You will learn how to display different RGB colors on your screen.

RGB stands for Red Green Blue. By mixing these three colors, we can create all kinds of colors on your screen. Most simply, if you put Red to its maximum of 255, and keep the other two at zero, then you will get a complete red screen.

In [ ]:
import cv2
import numpy as np

In [ ]:
red = 255
green = 0
blue = 0

current_color = (red, green, blue)

The following code will open up a red rectangle, for 3 seconds and then close:

In [ ]:
# Create a blank image
width, height = 800, 400
image = np.zeros((height, width, 3), dtype=np.uint8)

# Create a named window for display
cv2.namedWindow("Live Image", cv2.WINDOW_NORMAL)
cv2.startWindowThread()
image[:, :] = tuple(reversed(current_color)) # cv2 works with BGR order instead of RGB
# Display the image
cv2.imshow("Live Image", image)

cv2.waitKey(3000)  # Adjust the wait time (in milliseconds) as needed
# Close the CV2 window
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

**Task**: Now try it yourself: Change the RGB values above, and see what different rectangles you can create!

## (2) Characterisation: Using the Microbit with the Envirobit to measure RGB colors

**Goal**: You will be able to measure RGB colors with the Microbit
We will use the 'pyserial' library. Let's make sure it is installed

**Tasks**: 
1. Program the Microbit to measure RGB color on request.
2. Connect the Microbit to the computer, and make sure we request new measurements from it.

### (2.1) Programming the Microbit to send new RGB color measurements

Slot your Microbit into the Envirobit extension board (see [here](https://shop.pimoroni.com/products/enviro-bit)). Then, plug it into a USB socket on your computer and follow these steps:

1. Open MakeCode [here](https://makecode.microbit.org/#editor), in Google Chrome ([download](https://www.google.com/chrome/)).
2. In the middle column, somewhere between "Math" and "Variables" click on "Extensions".
3. Search for "envirobit", then click on it to load the envirobit library into your Microbit program. (it should appear below "Led" as "Enviro:Bit" with a teardrop icon).
4. Drag and drop the below program into your Microbit. The 'serial' blocks are hidden in the 'Advanced' section.

![Microbit Program](img/microbit_program.png "Microbit Program")

5. Alternatively, you can copy and paste the following code into the "JavaScript" section (switch to JavaScript via its button in the middle of the top bar):
```
serial.onDataReceived(serial.delimiters(Delimiters.Comma), function () {
    serial.writeLine(envirobit.getGreen() + "-" + envirobit.getRed() + "-" + envirobit.getBlue())
    music.playTone(262, music.beat(BeatFraction.Sixteenth))
})
music.setVolume(127)
```

6. Connect to your microbit by clicking 'Connect' on the bottom left. Follow the instructions.
7. Once connected, press download to load the program onto the microbit.


### (2.2) Connecting to the Microbit and requesting new measurements

**Warning**: Make 100% sure that you disconnected your Microbit from Google Chrome before continuing. Otherwise, Google Chrome will interfere with the serial connection and you cannot request values

Let's import the 'pyserial' package, which we use to connect to the Microbit (it is just called 'serial'):

In [ ]:
!pip install pyserial

In [ ]:
import serial

We will list all USB devices currently connected to our computer:

In [ ]:
!ls /dev/cu.*

Choose the one with 'cu.usbmodem' in its name and copy its name below:

In [ ]:
ser = serial.Serial()
ser=serial.Serial("/dev/cu.usbmodem102",115200, timeout=0.1)

In [ ]:
def characterise():
    valid = False
    while not valid:
        try:
            ser.flushInput()
            ser.write(b",")
            serial_data = str(ser.readline().decode('utf8')).rstrip()
            rgb = tuple([int(value) for value in serial_data.split("-")])
            # Measurement needs three entries
            if len(rgb) != 3:
                print(f"Measurement did not containt 3 RGB values:{rgb}")
                raise Exception()
            if not np.all([0 <= x <= 255 for x in rgb]):
                print(f"Measurement outside valid range [0,255]:{rgb}")
                raise Exception()
            valid = True
        except:
            print("Measurement invalid, will try again")
    print(f"New RGB Measurement: {rgb}")
    return rgb

In [ ]:
for i in range(10):
    characterise()

**Note**: To improve the connection between your computer and the microbit, make sure that Chrome is disconnected as otherwise it will keep sending message to the microbit. Most simply, close the Chrome application. We will not need it again anyway.

## (3) Planning: Using different search strategies: Random, Grid, Bayesian Optimisation

### (3.1) Generate a random color with CV2

In [ ]:
import random
import numpy as np
import cv2

# Function to generate random color
def generate_random_color():
    red = random.randint(0, 255)
    green = random.randint(0, 255)
    blue = random.randint(0, 255)
    return blue, green, red  # OpenCV uses BGR color format

# Function to calculate the error between two colors
def calculate_error(color1, color2):
    return np.sqrt((color1[0] - color2[0]) ** 2 + (color1[1] - color2[1]) ** 2 + (color1[2] - color2[2]) ** 2)

# Create a blank image
width, height = 800, 400
image = np.zeros((height, width, 3), dtype=np.uint8)

# Create a named window for display
cv2.namedWindow("Live Image", cv2.WINDOW_NORMAL)
cv2.startWindowThread()

# Define font properties for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_color = (255, 255, 255)  # White color

# Define the goal color
goal_color = (0, 0, 255)  # Red color (BGR format)

# Create an empty list to store the error values
error_values = []

# Main loop
max_iterations = 10  # Set the desired number of iterations

candidate_color = (0,0,0) # Initial candidate

for iteration in range(max_iterations):
    

    # STEP 1: Synthesis
    # Update the live image with the current color
    image[:, :] = candidate_color
    
    # Display the image
    cv2.imshow("Live Image", image)
    
    # STEP 2: Characterisation
    # Measure the color
    rgb_measurement = characterise()
    rgb_measurement = tuple(reversed(rgb_measurement))
    
    # STEP 3: Planning
    
    # Calculate the error between the goal color and the current color
    error = calculate_error(goal_color, rgb_measurement)

    # Add the error value to the list
    error_values.append(error)    

    # Add text information to the image
    text = f"Iteration: {iteration + 1}"
    cv2.putText(image, text, (10, 30), font, font_scale, font_color, 2)

    # Plot a subplot for the goal color
    # subplot_x = num_iterations * (width // (max_iterations + 1))
    subplot_x = width // (max_iterations + 1)
    subplot_width = width // (max_iterations + 1)
    cv2.rectangle(image, (subplot_x, 60), (subplot_x + subplot_width, height - 60), goal_color, -1)

    # Add text for the error
    error_text = f"Error: {error:.2f}"
    cv2.putText(image, error_text, (10, height - 10), font, font_scale, font_color, 2)

    # Draw the graph of error values
    if len(error_values) > 1:
        for i in range(1, len(error_values)):
            x1 = (i - 1) * (width // max_iterations)
            y1 = height - int(error_values[i - 1] * (height - 60) / max(error_values))
            x2 = i * (width // max_iterations)
            y2 = height - int(error_values[i] * (height - 60) / max(error_values))
            cv2.line(image, (x1, y1), (x2, y2), (255, 255, 255), 2)

    # Increment the iteration counter
    cv2.imshow("Live Image", image)
    cv2.waitKey(200)  # Adjust the wait time (in milliseconds) as needed
    
    # Generate a new candidate color, for now we will pick one randomly
    candidate_color = generate_random_color()
    
# Close the window after the desired number of iterations
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(error_values)

### (3.2) Use a search over a grid to find the best RGB combination

TBD Explanation

In [ ]:
grid_steps = 3
red = np.linspace(0, 255, grid_steps)
green = np.linspace(0, 255, grid_steps)
blue = np.linspace(0, 255, grid_steps)

grid_combinations = []
for r in red:
    for g in green:
        for b in blue:
            combination = (r,g,b)
            grid_combinations.append(combination)
grid_combinations

In [ ]:
import random
import numpy as np
import cv2

# Function to generate random color
def generate_random_color():
    red = random.randint(0, 255)
    green = random.randint(0, 255)
    blue = random.randint(0, 255)
    return blue, green, red  # OpenCV uses BGR color format

# Function to calculate the error between two colors
def calculate_error(color1, color2):
    return np.sqrt((color1[0] - color2[0]) ** 2 + (color1[1] - color2[1]) ** 2 + (color1[2] - color2[2]) ** 2)

# Create a blank image
width, height = 800, 400
image = np.zeros((height, width, 3), dtype=np.uint8)

# Create a named window for display
cv2.namedWindow("Live Image", cv2.WINDOW_NORMAL)
cv2.startWindowThread()

# Define font properties for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_color = (255, 255, 255)  # White color

# Define the goal color
goal_color = (0, 0, 255)  # Red color (BGR format)

# Create an empty list to store the error values
error_values = []

# Main loop
max_iterations = len(grid_combinations)  # Set the desired number of iterations

candidate_color = grid_combinations[0]

for iteration, grid_combination in enumerate(grid_combinations):
    

    # STEP 1: Synthesis
    # Update the live image with the current color
    image[:, :] = candidate_color
    
    # Display the image
    cv2.imshow("Live Image", image)
    
    # STEP 2: Characterisation
    # Measure the color
    rgb_measurement = characterise()
    rgb_measurement = tuple(reversed(rgb_measurement))
    
    # STEP 3: Planning
    
    # Calculate the error between the goal color and the current color
    error = calculate_error(goal_color, rgb_measurement)

    # Add the error value to the list
    error_values.append(error)    

    # Add text information to the image
    text = f"Iteration: {iteration + 1}"
    cv2.putText(image, text, (10, 30), font, font_scale, font_color, 2)

    # Plot a subplot for the goal color
    # subplot_x = num_iterations * (width // (max_iterations + 1))
    subplot_x = width // (max_iterations + 1)
    subplot_width = width // (max_iterations + 1)
    cv2.rectangle(image, (subplot_x, 60), (subplot_x + subplot_width, height - 60), goal_color, -1)

    # Add text for the error
    error_text = f"Error: {error:.2f}"
    cv2.putText(image, error_text, (10, height - 10), font, font_scale, font_color, 2)

    # Draw the graph of error values
    if len(error_values) > 1:
        for i in range(1, len(error_values)):
            x1 = (i - 1) * (width // max_iterations)
            y1 = height - int(error_values[i - 1] * (height - 60) / max(error_values))
            x2 = i * (width // max_iterations)
            y2 = height - int(error_values[i] * (height - 60) / max(error_values))
            cv2.line(image, (x1, y1), (x2, y2), (255, 255, 255), 2)

    # Increment the iteration counter
    cv2.imshow("Live Image", image)
    cv2.waitKey(100)  # Adjust the wait time (in milliseconds) as needed
    
    # Generate a new candidate color, for now we will pick one randomly
    candidate_color = grid_combination
    
# Close the window after the desired number of iterations
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
plt.plot(error_values)

### (3.3) Use Bayesian Optimisation to search for the optimal RGB combination

In [ ]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
from botorch.models.transforms.input import Normalize
from botorch.models.transforms.outcome import Standardize

In [ ]:
import pandas as pd
def calculate_candidate(samples):
    train_X = samples[["R","G","B"]]
    train_Y = samples[["error"]]

    train_X = torch.tensor(train_X.to_numpy(dtype=np.float64))
    train_Y = torch.tensor(-1*train_Y.to_numpy(dtype=np.float64))

    gp = SingleTaskGP(train_X, train_Y, input_transform=Normalize(d=train_X.shape[-1]), outcome_transform=Standardize(m=train_Y.shape[-1]),)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    from botorch.acquisition import UpperConfidenceBound
    UCB = UpperConfidenceBound(gp, beta=0.1)
    
    from botorch.optim import optimize_acqf
    bounds = torch.stack([torch.zeros(3), torch.ones(3)*255])
    candidate, acq_value = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
    )
    candidate = candidate[0]
    candidate = {"R": candidate[0], "G": candidate[1], "B": candidate[2]}
    return candidate

In [ ]:
import random
import numpy as np
import cv2

# Create a blank image
width, height = 800, 400
image = np.zeros((height, width, 3), dtype=np.uint8)

# Create a named window for display
cv2.namedWindow("Live Image", cv2.WINDOW_NORMAL)
cv2.startWindowThread()

# Define font properties for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_color = (255, 255, 255)  # White color

# Define the goal color
goal_color = (70,125,50) 

# Main loop
max_iterations = 50  # Set the desired number of iterations

# Let us set up a data table to record the data more structured
samples = pd.DataFrame(columns=['R', 'G', 'B', 'error', 'ID'])

candidate_color = (0,0,0) # Initial color

for iteration in range(max_iterations):
    print(f"---Iteration {iteration}")

    # STEP 1: Synthesis
    # Update the live image with the current color
    print(f"Synthesis:{candidate_color}")
    image[:, :] = tuple(reversed(candidate_color))
    
    # Display the image
    cv2.imshow("Live Image", image)
    
    # STEP 2: Characterisation
    # Measure the color by just plugging it in
    rgb_measurement = candidate_color
    print(f"Characterisation:{rgb_measurement}")
    
    # STEP 3: Planning
    # Calculate the loss between the goal color and the current color
    error = calculate_error(goal_color, rgb_measurement)
    print(f"Error:{error}")

    # Add the sample to the data table
    new_sample = pd.DataFrame({"R":rgb_measurement[0], "G":rgb_measurement[1], "B":rgb_measurement[2], "ID":iteration, "error":error}, index=[iteration])
    samples = pd.concat([samples, new_sample], axis=0, ignore_index=True)

    candidate = calculate_candidate(samples)
    candidate_color = (candidate['R'].item(), candidate['G'].item(), candidate['B'].item())
    print(f"Planning: Candidate:{candidate_color}")
    
    # Add text information to the image
    text = f"Iteration: {iteration + 1}"
    cv2.putText(image, text, (10, 30), font, font_scale, font_color, 2)

    # Plot a subplot for the goal color
    # subplot_x = num_iterations * (width // (max_iterations + 1))
    subplot_x = width // (max_iterations + 1)
    subplot_width = 50
    cv2.rectangle(image, (subplot_x, 60), (subplot_x + subplot_width, height - 60), tuple(reversed(goal_color)), -1)

    # Add text for the error
    error_text = f"Error: {error:.2f}"
    cv2.putText(image, error_text, (10, height - 10), font, font_scale, font_color, 2)

    error_values = list(samples['error'].to_numpy())    # Draw the graph of error values
    if len(error_values) > 1:
        for i in range(1, len(error_values)):
            x1 = (i - 1) * (width // max_iterations)
            y1 = height - int(error_values[i - 1] * (height - 60) / max(error_values))
            x2 = i * (width // max_iterations)
            y2 = height - int(error_values[i] * (height - 60) / max(error_values))
            cv2.line(image, (x1, y1), (x2, y2), (255, 255, 255), 2)

    # Increment the iteration counter
    cv2.imshow("Live Image", image)
    cv2.waitKey(100)  # Adjust the wait time (in milliseconds) as needed
    
# Close the window after the desired number of iterations
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
import matplotlib.pyplot as plt
# Plot error for each iteration
plt.scatter(range(len(error_values)), error_values, label="Error for each iteration")

# Calculate best error so far
best_error_so_far = 999999
best_errors = []
for value in error_values:
    if value < best_error_so_far:
        best_error_so_far = value
    best_errors.append(best_error_so_far)
    
# Plot best error so far
plt.plot(best_errors, label="Best Error So Far", color="orange")
plt.legend()

## (4) Congratulations! 

### You just ran your first closed-loop self-driving laboratory. You learned how to do its 3 core steps:
### 1. Synthesis: *Generating the material: a color on your screen (RGB)*
### 2. Charactersiation: *Measuring the material: the color on your screen (in RGB)*
### 3. Planning: *Choosing the next experiment: randomly, with a grid or with Bayesian Optimisation*

## Next Steps: We have prepared notebooks that discuss each step in detail, see https://github.com/matterhorn-studio/SDL4Kids.com